# Train Scheduler Visualization

This notebook provides interactive visualizations for:
- Railway tracks/routes
- Station positions
- Elevation profiles
- Network overview

In [ ]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, LineString
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Load Data

Load the output files from your Train Scheduler run

In [ ]:
# Update these paths to match your output directory
# Since the notebook is in the 'notebooks' folder, we need to go up one level
OUTPUT_DIR = "../output/"
INPUT_DIR = "../input/"

# Load city data
cities_df = pd.read_csv(f"{INPUT_DIR}lebanon_cities_2024.csv")
print(f"Loaded {len(cities_df)} cities")
cities_df.head()

In [ ]:
# Load route data (if available)
try:
    # Load GeoJSON tracks from output directory
    route_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('_tracks.geojson')]
    
    if route_files:
        # Combine all track files
        routes_list = []
        for file in route_files:
            gdf = gpd.read_file(os.path.join(OUTPUT_DIR, file))
            # Extract city code from filename (e.g., 'LB-BEY' from 'LB-BEY_tracks.geojson')
            city_code = file.replace('_tracks.geojson', '')
            gdf['city_code'] = city_code
            routes_list.append(gdf)
        
        routes_gdf = pd.concat(routes_list, ignore_index=True)
        print(f"Loaded {len(routes_gdf)} routes from {len(route_files)} files")
    else:
        raise FileNotFoundError("No track files found")
except Exception as e:
    print(f"Could not load route files: {e}")
    print("Creating sample routes from city pairs.")
    # Create sample routes between cities
    routes = []
    for i in range(len(cities_df) - 1):
        city1 = cities_df.iloc[i]
        city2 = cities_df.iloc[i + 1]
        route = {
            'origin': city1['city_name'],
            'destination': city2['city_name'],
            'geometry': LineString([
                (city1['longitude'], city1['latitude']),
                (city2['longitude'], city2['latitude'])
            ])
        }
        routes.append(route)
    routes_gdf = gpd.GeoDataFrame(routes, crs='EPSG:4326')

In [ ]:
# Load station data (if available)
try:
    stations_df = pd.read_csv(f"{OUTPUT_DIR}stations.csv")
    print(f"Loaded {len(stations_df)} stations")
except:
    print("No stations file found. Using cities as stations.")
    stations_df = cities_df.copy()
    stations_df['station_type'] = 'Terminal'  # Default station type

## 2. Create Interactive Map with Folium

In [ ]:
# Create base map centered on Lebanon
# Calculate center point
center_lat = cities_df['latitude'].mean()
center_lon = cities_df['longitude'].mean()

# Create folium map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=8,
    tiles='OpenStreetMap'
)

# Add different tile layers
folium.TileLayer('CartoDB dark_matter', name='Dark Mode').add_to(m)
folium.TileLayer('CartoDB positron', name='Light Mode').add_to(m)
folium.TileLayer('Stamen Terrain', name='Terrain').add_to(m)

In [ ]:
# Add cities to map
cities_layer = folium.FeatureGroup(name='Cities')

for idx, city in cities_df.iterrows():
    # Create popup text
    popup_text = f"""
    <b>{city['city_name']}</b><br>
    Population: {city.get('population', 'N/A'):,}<br>
    Lat: {city['latitude']:.4f}<br>
    Lon: {city['longitude']:.4f}
    """
    
    # Add city marker
    folium.CircleMarker(
        location=[city['latitude'], city['longitude']],
        radius=8,
        popup=folium.Popup(popup_text, max_width=200),
        color='darkblue',
        fill=True,
        fillColor='lightblue',
        fillOpacity=0.7,
        weight=2
    ).add_to(cities_layer)
    
    # Add city label
    folium.Marker(
        location=[city['latitude'], city['longitude']],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 10pt; color: black; font-weight: bold; text-shadow: 1px 1px 1px white;">{city["city_name"]}</div>'
        )
    ).add_to(cities_layer)

cities_layer.add_to(m)

In [ ]:
# Add railway routes to map
routes_layer = folium.FeatureGroup(name='Railway Routes')

# Define route colors based on type or priority
route_colors = ['red', 'blue', 'green', 'purple', 'orange']

for idx, route in routes_gdf.iterrows():
    # Get route coordinates
    if hasattr(route.geometry, 'coords'):
        coords = list(route.geometry.coords)
    else:
        coords = [(route.geometry.x, route.geometry.y)]
    
    # Flip coordinates for folium (expects lat, lon)
    coords_flipped = [(lat, lon) for lon, lat in coords]
    
    # Create popup text
    popup_text = f"""
    <b>Route {idx + 1}</b><br>
    From: {route.get('origin', 'Unknown')}<br>
    To: {route.get('destination', 'Unknown')}<br>
    Distance: {route.geometry.length * 111:.1f} km (approx)
    """
    
    # Add route line
    folium.PolyLine(
        locations=coords_flipped,
        color=route_colors[idx % len(route_colors)],
        weight=4,
        opacity=0.8,
        popup=folium.Popup(popup_text, max_width=200)
    ).add_to(routes_layer)
    
    # Add route animation (optional)
    folium.plugins.AntPath(
        locations=coords_flipped,
        color=route_colors[idx % len(route_colors)],
        weight=2,
        opacity=0.5
    ).add_to(routes_layer)

routes_layer.add_to(m)

In [ ]:
# Add stations to map
stations_layer = folium.FeatureGroup(name='Train Stations')

# Define station icons based on type
station_icons = {
    'Terminal': {'color': 'red', 'icon': 'fa-train', 'size': 12},
    'Through': {'color': 'blue', 'icon': 'fa-circle', 'size': 10},
    'Junction': {'color': 'green', 'icon': 'fa-code-fork', 'size': 11},
    'Rural': {'color': 'gray', 'icon': 'fa-circle-o', 'size': 8}
}

for idx, station in stations_df.iterrows():
    station_type = station.get('station_type', 'Terminal')
    icon_props = station_icons.get(station_type, station_icons['Terminal'])
    
    # Create popup text
    popup_text = f"""
    <b>{station.get('city_name', station.get('station_name', 'Station'))}</b><br>
    Type: {station_type}<br>
    Platforms: {station.get('platforms', 'N/A')}<br>
    Daily Passengers: {station.get('daily_passengers', 'N/A')}
    """
    
    # Add station marker
    folium.Marker(
        location=[station['latitude'], station['longitude']],
        popup=folium.Popup(popup_text, max_width=200),
        icon=folium.Icon(
            color=icon_props['color'],
            icon=icon_props['icon'],
            prefix='fa'
        )
    ).add_to(stations_layer)

stations_layer.add_to(m)

In [ ]:
# Add additional map features
# Add minimap
minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)

# Add fullscreen button
plugins.Fullscreen().add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add scale
m.add_child(folium.LatLngPopup())

# Save and display map
m.save('railway_network_map.html')
print("Map saved as 'railway_network_map.html'")
m

## 3. Network Analysis Visualizations

In [ ]:
# Create network statistics plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. City population distribution
ax1 = axes[0, 0]
if 'population' in cities_df.columns:
    cities_df.plot(kind='bar', x='city_name', y='population', ax=ax1, legend=False)
    ax1.set_title('Population by City')
    ax1.set_xlabel('City')
    ax1.set_ylabel('Population')
    ax1.tick_params(axis='x', rotation=45)

# 2. Route length distribution
ax2 = axes[0, 1]
route_lengths = [route.geometry.length * 111 for _, route in routes_gdf.iterrows()]
ax2.hist(route_lengths, bins=20, edgecolor='black')
ax2.set_title('Distribution of Route Lengths')
ax2.set_xlabel('Distance (km)')
ax2.set_ylabel('Number of Routes')

# 3. Station types distribution
ax3 = axes[1, 0]
if 'station_type' in stations_df.columns:
    station_counts = stations_df['station_type'].value_counts()
    station_counts.plot(kind='pie', ax=ax3, autopct='%1.1f%%')
    ax3.set_title('Station Types Distribution')
    ax3.set_ylabel('')

# 4. Network connectivity
ax4 = axes[1, 1]
# Calculate degree (number of connections) for each city
city_connections = {}
for city in cities_df['city_name']:
    connections = 0
    for _, route in routes_gdf.iterrows():
        if route.get('origin') == city or route.get('destination') == city:
            connections += 1
    city_connections[city] = connections

connectivity_df = pd.DataFrame(list(city_connections.items()), columns=['City', 'Connections'])
connectivity_df.plot(kind='bar', x='City', y='Connections', ax=ax4, legend=False)
ax4.set_title('Network Connectivity by City')
ax4.set_xlabel('City')
ax4.set_ylabel('Number of Connections')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('network_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Elevation Profile Visualization

In [ ]:
# Create elevation profile for routes
# This is a placeholder - actual elevation data would come from your DEM processing
fig, ax = plt.subplots(figsize=(15, 6))

# For each route, plot elevation profile
for idx, route in routes_gdf.iterrows():
    # Generate sample elevation data (replace with actual DEM data)
    route_length = route.geometry.length * 111  # km
    distance_points = np.linspace(0, route_length, 100)
    
    # Simulate elevation profile (replace with actual elevation data)
    base_elevation = 500 + idx * 100
    elevation_profile = base_elevation + 50 * np.sin(distance_points / 10) + \
                       20 * np.random.randn(len(distance_points))
    
    # Plot elevation profile
    ax.plot(distance_points, elevation_profile, 
            label=f"{route.get('origin', 'A')} → {route.get('destination', 'B')}",
            linewidth=2)

ax.set_xlabel('Distance (km)')
ax.set_ylabel('Elevation (m)')
ax.set_title('Railway Route Elevation Profiles')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('elevation_profiles.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Cost Analysis Visualization

In [ ]:
# Create cost breakdown visualization
# Sample cost data (replace with actual data from your analysis)
cost_categories = ['Land Acquisition', 'Construction', 'Stations', 'Bridges/Tunnels', 'Equipment']
costs = [150, 400, 200, 300, 100]  # Million USD

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart
ax1.pie(costs, labels=cost_categories, autopct='%1.1f%%', startangle=90)
ax1.set_title('Cost Breakdown by Category')

# Bar chart with cumulative line
ax2.bar(cost_categories, costs, color='skyblue', edgecolor='black')
ax2.set_ylabel('Cost (Million USD)')
ax2.set_title('Project Cost by Category')
ax2.tick_params(axis='x', rotation=45)

# Add cumulative cost line
cumulative_costs = np.cumsum(costs)
ax2_twin = ax2.twinx()
ax2_twin.plot(cost_categories, cumulative_costs, color='red', marker='o', linewidth=2)
ax2_twin.set_ylabel('Cumulative Cost (Million USD)', color='red')
ax2_twin.tick_params(axis='y', colors='red')

plt.tight_layout()
plt.savefig('cost_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Export Summary Report

In [ ]:
# Generate summary report
summary = f"""
# Railway Network Summary Report

## Network Overview
- Total Cities: {len(cities_df)}
- Total Routes: {len(routes_gdf)}
- Total Stations: {len(stations_df)}
- Total Track Length: {sum(route_lengths):.1f} km

## Station Distribution
{stations_df['station_type'].value_counts().to_string() if 'station_type' in stations_df.columns else 'N/A'}

## Route Statistics
- Average Route Length: {np.mean(route_lengths):.1f} km
- Shortest Route: {min(route_lengths):.1f} km
- Longest Route: {max(route_lengths):.1f} km

## Cost Estimate
- Total Estimated Cost: ${sum(costs)} Million USD
- Cost per km: ${sum(costs) / sum(route_lengths):.2f} Million USD/km

## Files Generated
1. railway_network_map.html - Interactive map
2. network_analysis.png - Network statistics
3. elevation_profiles.png - Route elevation profiles
4. cost_analysis.png - Cost breakdown charts
"""

print(summary)

# Save summary to file
with open('network_summary.txt', 'w') as f:
    f.write(summary)

print("\n✅ Visualization complete! Check the generated files.")